In [1]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import our input dataset using EIN as index
charity_df = pd.read_csv("charity_data.csv", index_col="EIN")
charity_df.head()

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
EIN,,,,,,,,,,,
10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Store the names of all charities
charity_names = charity_df['NAME']
charity_names_df = pd.DataFrame(data=charity_names, index=charity_df.index, columns=["NAME"])
charity_names_df.head()

,NAME
EIN,
10520599,BLUE KNIGHTS MOTORCYCLE CLUB
10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR
10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS
10553066,SOUTHSIDE ATHLETIC ASSOCIATION
10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT


In [3]:
# Remove Name column
charity_df.drop(columns=["NAME"], inplace=True)
charity_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
EIN,,,,,,,,,,
10520599,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
10531628,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
10547893,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
10553066,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
10556103,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Generate our categorical variable list
charity_cat = charity_df.dtypes[charity_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
charity_df[charity_cat].nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

In [5]:
#%%
# Check the unique value counts to see if binning is needed
classification_counts = charity_df.CLASSIFICATION.value_counts()
classification_counts.plot.density()

In [6]:
# Determine which values to replace
replace_classifications = list(classification_counts[classification_counts < 1000].index)

# Replace in DataFrame
for classification in replace_classifications:
    charity_df.CLASSIFICATION = charity_df.CLASSIFICATION.replace(classification, "Other")

# Check to make sure binning was successful
charity_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [7]:
# Check the unique value counts to see if binning is needed
apptype_counts = charity_df.APPLICATION_TYPE.value_counts()
apptype_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Determine which values to replace
replace_apptypes = list(apptype_counts[apptype_counts < 700].index)

# Replace in DataFrame
for apptype in replace_apptypes:
    charity_df.APPLICATION_TYPE = charity_df.APPLICATION_TYPE.replace(apptype, "Other")
    
# Check to make sure binning was successful
charity_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: APPLICATION_TYPE, dtype: int64